In [1]:
import plotly
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets

In [2]:
jd = pd.read_csv('./data/biblio_data.csv', sep=";", header=0)
alt = pd.read_csv('./data/cleaned_altmetrics.csv', sep=";", header=0)

In [6]:
data_grouped = alt.groupby('journal_name').agg(
    max=pd.NamedAgg(column='altmetric_score', aggfunc=max),
    min=pd.NamedAgg(column='altmetric_score', aggfunc=min),
    mean=pd.NamedAgg(column='altmetric_score', aggfunc=np.mean),
    median=pd.NamedAgg(column='altmetric_score', aggfunc=np.median),
)
data_grouped.reset_index(level=0, inplace=True)
data_grouped = data_grouped


In [4]:

select = widgets.Dropdown(
    options=[('Max', 'max'), ('Min', 'min'), ('Mean', 'mean'), ('Median', 'median')],
    value='max',
    description='Type:',
    disabled=False,
)

In [ ]:

@widgets.interact
def graph(Options=[('Max', 'max'), ('Min', 'min'), ('Mean', 'mean'), ('Median', 'median')]):
    fig = go.Figure(go.Bar(
                x=data_grouped[Options],
                y=data_grouped["journal_name"],
                orientation='h'))

    # add vertival line for average
    avg = np.mean(data_grouped[Options])
    fig.update_layout(shapes=[
        dict(
        type= 'line',
        yref= 'paper', y0= 0, y1= 1,
        xref= 'x', x0= avg, x1= avg,
        line=dict(color="red")
        )
    ])

    fig.update_layout(title='Altmetric Score per Journal',
                    xaxis_title='Altmetric Score',
                    yaxis_title='Journal Name')
    return fig.show()



In [ ]:
sources = ['news', 'blog', 'policy', 'patent', 'twitter', 'peer_review', 'weibo', 'facebook', 'wikipedia', 'google','linkedIn', 'reddit', 'pinterest', 'f1000', 'qa', 'videos', 'syllabi', 'mendeley']

data = pd.pivot_table(alt, index="journal_name", values=sources, aggfunc=np.sum)
data.reset_index(level=0, inplace=True)
labels = data.columns[1:].tolist()

filters = widgets.SelectMultiple(
        options=labels,
        value=labels[0::3],
        rows=10,
        description='Fruits',
        disabled=False
    )

@widgets.interact
def pie(Journals=data_grouped["journal_name"].tolist(), Filters=filters):
    selected_filters = Filters
    selected_journal = Journals
    row = data.loc[data["journal_name"] == selected_journal]
    d = row.T
    d = d.drop(d.index[[0]])
    d.reset_index(level=0, inplace=True)
    d.columns = ['labels', 'values']
    d.values
    fig = go.Figure(data=[go.Pie(labels=[x for x in d.labels if x in selected_filters], values=[x[1] for x in d.values if x[0] in selected_filters])])
    fig.update_layout(title=selected_journal)
    return fig.show()